In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
# print(torch.is_gpu_)

In [3]:

# Read the pickle object
with open("../../dataset/pickles/LOC3.pickle", "rb") as f:
    data = pickle.load(f)

# Convert the data to a pandas dataframe
df = pd.DataFrame(data)

# Encode the class labels
label_encoder = LabelEncoder()
df['class_label'] = label_encoder.fit_transform(df['class_label'])

# Define the maximum sequence length
max_len = 0
for i in df['lengths']:
    max_len=max(max_len,len(i))

# Pad the sequences to the maximum length
sequences = np.array(df['lengths'])
padded_sequences = np.zeros((len(sequences), max_len))
for i, sequence in enumerate(sequences):
    padded_sequences[i, :len(sequence)] = sequence

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, df['class_label'], test_size=0.2)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).to(torch.float32)
y_train = torch.from_numpy(y_train.values).long()
X_val = torch.from_numpy(X_val).to(torch.float32)
y_val = torch.from_numpy(y_val.values).long()
# check if split is stratified
# print("Training set class distribution:")
# print(y_train.unique(return_counts=True)) 
# print("Validation set class distribution:")
# print(y_val.unique(return_counts=True))
print(y_train.unique().shape)
print(y_val.unique().shape)

torch.Size([1500])
torch.Size([1500])


In [4]:
# Define a custom dataset
class SequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        sequence = self.sequences[index]
        label = self.labels[index]
        return sequence, label


In [5]:

# Initialize the model
input_dim=0
for i in X_train:
    input_dim = max(input_dim,len(i))
# input_dim = len(X_train[0])
hidden_dim = 128
output_dim = 1500
learning_rate = 1e-3
print(input_dim)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
# Define the dataloaders
train_dataset = SequenceDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = SequenceDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


1213


In [6]:

# Define the model architecture
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, batch_size):
        super(RNN, self).__init__()
        self.batch_size=batch_size
        # simple 1d conv 
        self.conv1 = nn.Conv1d(1, 8, 3, padding=1)
        self.bn1 = nn.BatchNorm1d(8)
        self.relu = nn.ReLU()        
        
        # parameters for the RNN
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.output_dim = output_dim

        # define the RNN
        # self.rnn = nn.LSTM(8*input_dim, hidden_dim, batch_first=True)
        self.rnn = nn.RNN(8*input_dim, hidden_dim, batch_first=True)
        # define the output layer
        self.fc = nn.Linear(hidden_dim, output_dim)    
        self.h = self.init_hidden(self.batch_size)    
        self.flatten = nn.Flatten()
        # self.fc = nn.Linear(8*input_dim, output_dim)
        
    def init_hidden(self, bsz):
        # initialize the hidden state
        self.h=torch.zeros(1, bsz, self.hidden_dim).to(device)
    
    def forward(self, x):
        # reshape the input
        # x = x.view(self.batch_size, self.input_dim, -1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.flatten(x)
        # initialize the hidden state
        self.init_hidden(x.shape[0])
        # get RNN unit outputs
        out, h = self.rnn(x.unsqueeze(1), self.h)
        # get the output for the last time step
        out = self.fc(out[:, -1, :])
        return out

In [7]:
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
rnn = RNN(input_dim, hidden_dim, output_dim, batch_size=64)
rnn = rnn.to(device)
criterion = criterion.to(device)

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)


Using device: cuda:6


In [8]:
from sklearn.metrics import precision_recall_fscore_support
def get_precision_recall_f1(y_true, y_pred):
    """Calculate precision, recall, F1 score for each class.
    Args:
        y_true (list): list of true labels
        y_pred (list): list of predicted labels
    Returns:
        dict: dictionary with precision, recall, F1 score for each class
    """
    # Calculate precision, recall, F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

num_epochs = 50
# optimizer = optimizer.to(device)
for epoch in range(num_epochs):
    running_loss = 0.0
    running_corrects = 0
    rnn.train()
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        labels = labels.to(device)
        sequences = sequences.to(device)
        sequences = sequences.unsqueeze(1)
        outputs = rnn(sequences)
        
        # print(sequences.shape)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() /len(train_dataset)
    print('Train Loss: {:.4f} Train Acc: {:.4f} %'.format(epoch_loss, epoch_acc*100))
    
    # test the model
    rnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        total_preds = []
        total_labels = []
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            sequences = sequences.unsqueeze(1)
            outputs = rnn(sequences)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_preds.append(predicted)
            total_labels.append(labels)
        
        print('Epoch: {}, Test Accuracy of the model on the test sequences: {} %'.format(epoch, 100 * correct / total))
        total_preds = torch.cat(total_preds, dim=0)
        total_labels = torch.cat(total_labels, dim=0)
        print(get_precision_recall_f1(total_labels.cpu(), total_preds.cpu()))

Train Loss: 4.2927 Train Acc: 18.7041 %
Epoch: 0, Test Accuracy of the model on the test sequences: 36.27004873011263 %
{'precision': 0.3719240144743547, 'recall': 0.36735669044085306, 'f1': 0.32531710261567903}


/home/anirudha/miniconda3/envs/cod/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 2.3597 Train Acc: 47.0761 %
Epoch: 1, Test Accuracy of the model on the test sequences: 53.306225191209215 %
{'precision': 0.5381971276026251, 'recall': 0.5335120526474874, 'f1': 0.5067731308629632}


/home/anirudha/miniconda3/envs/cod/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 1.7119 Train Acc: 60.1520 %
Epoch: 2, Test Accuracy of the model on the test sequences: 61.70329493013199 %
{'precision': 0.6295597335609329, 'recall': 0.6188084273709336, 'f1': 0.6005461698545397}


...

/home/anirudha/miniconda3/envs/cod/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 0.3084 Train Acc: 90.0045 %
Epoch: 48, Test Accuracy of the model on the test sequences: 81.74718430309484 %
{'precision': 0.8263390971586377, 'recall': 0.8180449800287127, 'f1': 0.8108010770695252}


/home/anirudha/miniconda3/envs/cod/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 0.3035 Train Acc: 90.1990 %
Epoch: 49, Test Accuracy of the model on the test sequences: 81.97308548746248 %
{'precision': 0.8275994097483517, 'recall': 0.8210039022559237, 'f1': 0.8149194164143234}


/home/anirudha/miniconda3/envs/cod/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
# save model
torch.save(rnn.state_dict(), 'rnn_model_.ckpt')


In [ ]:
print(total_preds.unique().shape)
print(total_labels.unique().shape)
print("Accuracy:{:.4f}".format(torch.sum(total_preds == total_labels).item()/len(total_labels)))
print(total_preds.shape)
print("Precision:{:.4f} Recall:{:.4f} F1:{:.4f}".format(get_precision_recall_f1(total_labels.cpu(), total_preds.cpu())['precision'], get_precision_recall_f1(total_labels.cpu(), total_preds.cpu())['recall'], get_precision_recall_f1(total_labels.cpu(), total_preds.cpu())['f1']))

In [ ]:
# Evaluate the model
running_loss = 0.0
running_corrects = 0
rnn.eval()
with torch.no_grad():
    for sequences, labels in val_loader:
        labels = labels.to(device)
        sequences = sequences.to(device)
        sequences = sequences.unsqueeze(1)
        outputs = rnn(sequences)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * sequences.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)
epoch_loss = running_loss / len(val_dataset)
epoch_acc = running_corrects.double() / len(val_dataset)
print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss, epoch_acc))


In [ ]:
accuracy = model.predict(test_data)
print("Accuracy: ", accuracy,"%")

In [ ]:
a = torch.tensor([[1]])